# This is way too slow
Even the LombScargleMultibandFast takes ~1.5h for train set (it uses all 20 cores so there's no point in distributing). This would take >20 days to process the test set which is obviously too much time.

In [ ]:
import warnings
from itertools import chain
import multiprocessing as mp

from tqdm.autonotebook import tqdm
from numba import jit

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', RuntimeWarning)

from gatspy.periodic import LombScargleMultiband, LombScargleMultibandFast

In [ ]:
train_series = pd.read_csv('../data/raw/training_set.csv')

In [ ]:
train_series.head()

In [ ]:
@jit
def fit_multiband_freq(gbo):
    idx, group = gbo
    t, f, e, b = group['mjd'], group['flux'], group['flux_err'], group['passband']
    model = LombScargleMultibandFast(fit_period=True, optimizer_kwds={'quiet': True})
    model.optimizer.period_range = (0.1, int((group['mjd'].max() - group['mjd'].min()) / 2))
    model.fit(t, f, e, b)
    return model

In [ ]:
# train_series_sample = train_series[:int(len(train_series) / 100)]

In [ ]:
%%time
warnings.simplefilter('ignore', UserWarning)  # ignore too short dataset warning from Fast impl. of LombScargle
models = [fit_multiband_freq(gbo) for gbo in tqdm(train_series.groupby('object_id'))]

In [ ]:
%%time
# this should not take more than 5 mins (on 12 cores)
warnings.simplefilter('ignore', UserWarning)  # ignore too short dataset warning from Fast impl. of LombScargle
with mp.Pool(12) as p:
    models = p.map(fit_multiband_freq, train_series.groupby('object_id'))

In [ ]:
feats = pd.DataFrame({
    'period': [m.best_period for m in models],
    'period_score': [m.score(m.best_period) for m in models],
}, index=train_series['object_id'].unique())

In [ ]:
feats.head(10)